In [1]:
!pip install openai
!pip install llama-index


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 57.6/57.6 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 15.4/15.4 MB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 2.0/2.0 MB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 371.6/371.6 kB 3.8 MB/s eta 0:00:00
   ---------------------------------------- 147.9/147.9 kB 2.9 MB/s eta 0:00:00
   ---------------------------------------- 176.9/176.9 kB 3.5 MB/s eta 0:00:00
   ---------------------------------------- 130.8/130.8 kB 7.5 MB/s eta 0:00:00
   ---------------------------------------- 1.6/1.6 MB 4.0 MB/s eta 0:00:00
   ---------------------------------------- 1.5/1.5 MB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 327.4/327.4 kB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 290.4/290.4 kB 3.6 MB/s eta 0:00:00
   ---------------------------------------- 152.8/152.8 kB 4.6 MB/s eta 0:00:00
   ---------------------------------------- 64.9/64.9 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.
conda-repo-cli 1.0.24 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.24 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.
conda-repo-cli 1.0.24 requires PyYAML==6.0, but you have pyyaml 6.0.1 which is incompatible.
conda-repo-cli 1.0.24 requires requests==2.28.1, but you have requests 2.32.3 which is incompatible.
langgraph 0.0.69 requires langchain-core<0.3,>=0.2, but you have langchain-core 0.0.13 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
import os
openAI_API_Key=os.environ['openAI_API_Key']

In [6]:
import nest_asyncio

nest_asyncio.apply()
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["D:\\University\\Test\\1.pdf"]).load_data()

In [7]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

The following setting is optional:

In [8]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [9]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)

In [10]:
vector_index = VectorStoreIndex(nodes)

**SummaryIndex**


The SummaryIndex class is typically used to create an index that summarizes the content of the nodes. This kind of index can be useful for applications where you need a quick overview or summary of large documents or collections of text. The summary might be generated by combining or condensing the information in the nodes.

**VectorStoreIndex**

The VectorStoreIndex class creates an index that stores vector representations of the nodes. These vectors can be used for various purposes, such as semantic search, similarity matching, and other machine learning tasks. The vector representations are typically created using techniques like word embeddings or sentence embeddings, which transform text into high-dimensional vectors that capture the semantic meaning of the text.

Define Query Engines and Set Metadata:

Note:A query engine in the context of llama_index (and similar libraries) is a component that allows you to query an index and retrieve relevant information based on the query. It provides an interface to interact with the indexed data, process queries, and return results.

The as_query_engine method essentially configures the index to be queried efficiently. It sets up the necessary components and parameters to handle queries, process them, and return appropriate results.

In [13]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

response_mode="tree_summarize": Specifies the mode for generating responses. In this case, it uses a tree summarization method, which likely organizes and summarizes the content in a hierarchical manner.


use_async=True: Indicates that the query engine should operate asynchronously, which can be useful for handling multiple queries concurrently or for improving performance in an asynchronous environment.

vector_index.as_query_engine
Purpose: Creates a query engine for the VectorStoreIndex.
Default Parameters: This method does not specify any additional parameters, so it uses the default settings for the query engine.

Next we should set our query (prompt) to be quried in the context.

In [23]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to TRIZ and its integration"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the TRIZ."
    ),
)

In [19]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

The RouterQueryEngine is designed to coordinate and route queries to appropriate tools or query engines (summary_tool and vector_tool in this case). The LLMSingleSelector helps in selecting the appropriate tool based on predefined configurations or defaults.

In [20]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: The document seems to focus on summarization questions related to TRIZ and its integration..
The document discusses the application of Lean Six Sigma and TRIZ methodology in banking services, focusing on improving customer satisfaction, cost efficiency, and process speed. It highlights the integration of TRIZ into the Lean Six Sigma approach to enhance problem-solving in the service industry, particularly in banking services. The case study presented demonstrates the successful implementation of Lean Six Sigma with TRIZ in a savings bank company, resulting in significant improvements in process capability, waiting time reduction, and cost savings. The document emphasizes the importance of investing resources, training employees, and committing to the LSS approach to achieve successful business improvement in service operations.


In [17]:
print(len(response.source_nodes))

2


In [26]:
response = query_engine.query(
    "why should we integrate TRIZ and six sigma? I mean which part of TRIZ is weak that we should integrate it wiith six sigma?"
)
print(str(response))

Selecting query engine 0: TRIZ is known for its problem-solving and innovation techniques, but it may lack the structured approach and data-driven methodology that Six Sigma provides. Integrating TRIZ with Six Sigma can enhance problem-solving capabilities by combining creativity with statistical analysis..
TRIZ methodology is known for its effectiveness in analyzing customer needs and developing innovative solutions to meet those needs. However, in certain areas, such as the service industry, TRIZ can face challenges due to the abstract nature of service contradictions. By integrating TRIZ with Six Sigma, which is focused on process improvement and reducing variation, the weaknesses of TRIZ in addressing service industry problems can be complemented. This integration allows for a more comprehensive approach to problem-solving by combining TRIZ's innovative thinking with Six Sigma's data-driven and systematic methodology.
